In [1]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import json

In [2]:
hostname='localhost'
port='9094'

In [57]:
producer = KafkaProducer(bootstrap_servers=[hostname+':'+port])

In [58]:
future = producer.send('test', b'raw_bytes')

KeyboardInterrupt: 

In [39]:
# Asynchronous by default
future = producer.send('quotes_stream', b'raw_bytes')
future = producer.send('quotes', b'raw_bytes1')

KeyboardInterrupt: 

In [59]:
# encode objects{'code': '5199','buy_queue_vol':1000} via msgpack
m= {'code': '5199','buy_queue_vol':1000}
producer = KafkaProducer(bootstrap_servers=[hostname+':'+port],
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))

In [60]:
producer.send('test', m)

KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.

In [21]:
# Block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    # Decide what to do if produce request failed...
    log.exception()
    pass

# Successful result returns assigned partition and offset
print (record_metadata.topic)
print (record_metadata.partition)
print (record_metadata.offset)

# produce keyed messages to enable hashed partitioning
#producer.send('my-topic', key=b'foo', value=b'bar')

# encode objects via msgpack
#producer = KafkaProducer(value_serializer=msgpack.dumps)
#producer.send('msgpack-topic', {'key': 'value'})

# produce json messages
producer = KafkaProducer(bootstrap_servers=[hostname+':'+port],
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))
producer.send('json-topic', {'key': 'value'})

# produce asynchronously
for _ in range(100):
    producer.send('my-topic', m)

def on_send_success(record_metadata):
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)
    # handle exception

# produce asynchronously with callbacks
producer.send('my-topic', b'raw_bytes').add_callback(on_send_success).add_errback(on_send_error)

# block until all async messages are sent
producer.flush()

# configure multiple retries
producer = KafkaProducer(retries=5)

quotes
0
322


TypeError: Object of type 'bytes' is not JSON serializable